In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
import numpy as np

<built-in method __dir__ of sqlite3.Connection object at 0x7effa9f10a40>


In [2]:
symbol = 'BTCUSDT'
interval = '1h'
start_amount_invested = 100.0
target_margin = 1.0
p_ema = 200
min_rsi = 28
max_rsi = 80
stop_loss_multiplier = 5

In [3]:
data = utils.get_data(
    symbol=symbol,
    interval=interval,
    updata_data_from_web=False)
data = utils.truncate_data_in_days(data, myenv.days_to_validate_train)
data

,open_time,close
ix_open_time,,
2023-08-16 22:00:00,2023-08-16 22:00:00,28910.000000
2023-08-16 23:00:00,2023-08-16 23:00:00,28730.509766
2023-08-17 00:00:00,2023-08-17 00:00:00,28580.849609
2023-08-17 01:00:00,2023-08-17 01:00:00,28632.689453
2023-08-17 02:00:00,2023-08-17 02:00:00,28584.349609
...,...,...
2023-11-14 18:00:00,2023-11-14 18:00:00,35230.000000
2023-11-14 19:00:00,2023-11-14 19:00:00,35293.988281
2023-11-14 20:00:00,2023-11-14 20:00:00,35285.988281


In [4]:
data = calc_utils.calc_RSI(data)
data

,open_time,close,rsi
ix_open_time,,,
2023-08-16 22:00:00,2023-08-16 22:00:00,28910.000000,NaN
2023-08-16 23:00:00,2023-08-16 23:00:00,28730.509766,NaN
2023-08-17 00:00:00,2023-08-17 00:00:00,28580.849609,NaN
2023-08-17 01:00:00,2023-08-17 01:00:00,28632.689453,NaN
2023-08-17 02:00:00,2023-08-17 02:00:00,28584.349609,NaN
...,...,...,...
2023-11-14 18:00:00,2023-11-14 18:00:00,35230.000000,24.147881
2023-11-14 19:00:00,2023-11-14 19:00:00,35293.988281,26.190498
2023-11-14 20:00:00,2023-11-14 20:00:00,35285.988281,26.095886


In [5]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[200])
data.dropna(inplace=True)
data

,open_time,close,rsi,ema_200p,ema_200p_diff
ix_open_time,,,,,
2023-08-25 05:00:00,2023-08-25 05:00:00,26059.990234,42.019966,26238.267578,-0.679455
2023-08-25 06:00:00,2023-08-25 06:00:00,26015.369141,39.490547,26235.705078,-0.839832
2023-08-25 07:00:00,2023-08-25 07:00:00,25980.500000,37.586479,26232.777344,-0.961688
2023-08-25 08:00:00,2023-08-25 08:00:00,26093.509766,46.576859,26231.183594,-0.524848
2023-08-25 09:00:00,2023-08-25 09:00:00,26079.980469,45.727634,26229.453125,-0.569866
...,...,...,...,...,...
2023-11-14 18:00:00,2023-11-14 18:00:00,35230.000000,24.147881,36311.804688,-2.979209
2023-11-14 19:00:00,2023-11-14 19:00:00,35293.988281,26.190498,36301.675781,-2.775871
2023-11-14 20:00:00,2023-11-14 20:00:00,35285.988281,26.095886,36291.570312,-2.770842


In [6]:
#data = utils.predict_strategy_index_all_data(data, p_ema, min_rsi, max_rsi)
#data

In [7]:
#data = utils.calc_take_profit_stop_loss_index(data, target_margin, stop_loss_multiplier)
#data

In [9]:
strategy = utils.predict_strategy_index(data, p_ema, max_rsi, min_rsi)
print(strategy)

ESTAVEL


In [8]:
for _, row in data.iterrows():
	print(type(row) == pd.core.series.Series)
	strategy = utils.predict_strategy_index(row, p_ema, max_rsi, min_rsi)
	print(strategy)

True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL
True
ESTAVEL

In [8]:
_data = data

purchased = False
perform_sell = False
purchase_price = 0.0
purchase_strategy = ''
label_ema = f'ema_{p_ema}p'
start_amount_invested = 100.0

print(f'Start Simule Trading: {symbol}_{interval} - Shape: {_data.shape} - Amount Invested: {start_amount_invested:.2f} - Target Margin: {target_margin}% - EMA: {p_ema}p - Min RSI: {min_rsi}% - Max RSI: {max_rsi}% - Stop Loss Multiplier: {stop_loss_multiplier}')

for _, row in data.iterrows():
	#_auxdata = _data.iloc[row_nu:row_nu + 1]
	
	actual_price = row['close']
	rsi = row['rsi']
	p_ema_value = row[f'ema_{p_ema}p']
	open_time = row['open_time']
	#strategy = row['strategy']
	#take_profit = row['take_profit']
	#stop_loss = row['stop_loss']


	if isinstance(open_time, np.datetime64):
		_open_time = pd.to_datetime(open_time, unit='ms').strftime('%Y-%m-%d %H:%M:%S')
	else:
		_open_time = open_time.strftime('%Y-%m-%d %H:%M:%S')
	
	strategy = utils.predict_strategy_index(row, p_ema, max_rsi, min_rsi)
	print(f'[{_open_time}][{strategy}] => Purchased: {purchased} - Price: {actual_price:.6f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi} - Max RSI: {max_rsi}')
	if not purchased:
		if strategy.startswith('LONG') or strategy.startswith('SHORT'):  # If true, BUY
			purchased = True
			purchase_price = actual_price
			purchase_strategy = strategy
			take_profit, stop_loss = utils.calc_take_profit_stop_loss(strategy, purchase_price, target_margin, stop_loss_multiplier)
			print(f'[{_open_time}] BUY: {purchase_strategy} - Price: {actual_price:.6f} - Target Margin: {target_margin:.2f}% - Take Profit: \
{take_profit:.2f} - Stop Loss: {stop_loss:.2f} - RSI: {rsi:.2f}% - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi}% - Max RSI: {max_rsi}% - Stop Loss Multiplier: {stop_loss_multiplier}')
			continue

	if purchased:  # and (operation.startswith('LONG') or operation.startswith('SHORT')):
		# print(f'[{row_nu}][{strategy}] => Purchased: {purchased} - Price: {actual_price:.6f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi} - Max RSI: {max_rsi}')
		if purchase_strategy.startswith('LONG'):
			margin_operation = (actual_price - purchase_price) / purchase_price
			if ((actual_price >= take_profit) or (actual_price <= stop_loss)):  # Long ==> Sell - Take Profit / Stop Loss
				perform_sell = True
		elif purchase_strategy.startswith('SHORT'):
			margin_operation = (purchase_price - actual_price) / purchase_price
			if ((actual_price <= take_profit) or (actual_price >= stop_loss)):  # Short ==> Sell - Take Profit / Stop Loss
				perform_sell = True

		if perform_sell:
			start_amount_invested = (1 + margin_operation) * start_amount_invested
			print(f'[{_open_time}] SELL: {purchase_strategy} - Price: {actual_price:.6f} - Purchase Price: {purchase_price:.6f} - Target Margin: {target_margin:.2f}% - Margin Operation:\
{margin_operation*100:.2f}% - Take Profit: {take_profit:.2f} - Stop Loss: {stop_loss:.2f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Sum PnL: {start_amount_invested:.2f}  - Stop Loss Multiplier: {stop_loss_multiplier}')
			perform_sell = False
			purchase_strategy = ''
			purchased = False

print(f'{symbol}_{interval} - Result Simule Trading: {start_amount_invested:.2f}')

Start Simule Trading: BTCUSDT_1h - Shape: (1962, 5) - Amount Invested: 100.00 - Target Margin: 1.0% - EMA: 200p - Min RSI: 28% - Max RSI: 80% - Stop Loss Multiplier: 5
(5,)


KeyError: 'close'

In [ ]:
_data = data

def calc():
	purchased = False
	perform_sell = False
	purchase_price = 0.0
	purchase_strategy = ''
	label_ema = f'ema_{p_ema}p'
	start_amount_invested = 100.0

	print(f'Start Simule Trading: {symbol}_{interval} - Shape: {_data.shape} - Amount Invested: {start_amount_invested:.2f} - Target Margin: {target_margin}% - EMA: {p_ema}p - Min RSI: {min_rsi}% - Max RSI: {max_rsi}% - Stop Loss Multiplier: {stop_loss_multiplier}')

	for _, row in data.iterrows():
		#_auxdata = _data.iloc[row_nu:row_nu + 1]
		
		actual_price = row['close']
		rsi = row['rsi']
		p_ema_value = row[f'ema_{p_ema}p']
		open_time = row['open_time']
		strategy = row['strategy']
		take_profit = row['take_profit']
		stop_loss = row['stop_loss']

		if isinstance(open_time, np.datetime64):
			_open_time = pd.to_datetime(open_time, unit='ms').strftime('%Y-%m-%d %H:%M:%S')
		else:
			_open_time = open_time.strftime('%Y-%m-%d %H:%M:%S')

		if not purchased:
			# strategy = utils.predict_strategy_index(_auxdata, p_ema, max_rsi, min_rsi)
			# print(f'[{row_nu}][{strategy}] => Purchased: {purchased} - Price: {actual_price:.6f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi} - Max RSI: {max_rsi}')
			if strategy.startswith('LONG') or strategy.startswith('SHORT'):  # If true, BUY
				purchased = True
				purchase_price = actual_price
				purchase_strategy = strategy
				#take_profit, stop_loss = utils.calc_take_profit_stop_loss(strategy, purchase_price, target_margin, stop_loss_multiplier)
	#			print(f'BUY[{_open_time}]: {purchase_strategy} - Price: {actual_price:.6f} - Target Margin: {target_margin:.2f}% - Take Profit: \
	#{take_profit:.2f} - Stop Loss: {stop_loss:.2f} - RSI: {rsi:.2f}% - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi}% - Max RSI: {max_rsi}% - Stop Loss Multiplier: {stop_loss_multiplier}')
				continue

		if purchased:  # and (operation.startswith('LONG') or operation.startswith('SHORT')):
			# print(f'[{row_nu}][{strategy}] => Purchased: {purchased} - Price: {actual_price:.6f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi} - Max RSI: {max_rsi}')
			if purchase_strategy.startswith('LONG'):
				margin_operation = (actual_price - purchase_price) / purchase_price
				if ((actual_price >= take_profit) or (actual_price <= stop_loss)):  # Long ==> Sell - Take Profit / Stop Loss
					perform_sell = True
			elif purchase_strategy.startswith('SHORT'):
				margin_operation = (purchase_price - actual_price) / purchase_price
				if ((actual_price <= take_profit) or (actual_price >= stop_loss)):  # Short ==> Sell - Take Profit / Stop Loss
					perform_sell = True

			if perform_sell:
				start_amount_invested = (1 + margin_operation) * start_amount_invested
				#print(f'SELL[{_open_time}]: {purchase_strategy} - Price: {actual_price:.6f} - Purchase Price: {purchase_price:.6f} - Target Margin: {target_margin:.2f}% - Margin Operation:\
	#{margin_operation*100:.2f}% - Take Profit: {take_profit:.2f} - Stop Loss: {stop_loss:.2f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Sum PnL: {start_amount_invested:.2f}  - Stop Loss Multiplier: {stop_loss_multiplier}')
				perform_sell = False
				purchase_strategy = ''
				purchased = False

	print(f'{symbol}_{interval} - Result Simule Trading: {start_amount_invested:.2f}')